# 凯恩斯《通论》数学模型 - 第二章：古典经济学假设

本笔记本整合了第二章的所有代码，展示古典经济学假设的数学模型和可视化。

In [ ]:
# 导入必要的库
import numpy as np
import matplotlib.pyplot as plt
import os
import math

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['Microsoft YaHei', 'SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

## 1. 工具函数模块 (来自 chapter_2_setup.py)

这部分保留为独立的 *_setup.py 文件，作为基础工具模块。

In [ ]:
# 由于 chapter_2_setup.py 是例外，我们在这里定义其核心函数

def cobb_douglas(K, L, alpha):
    """Cobb-Douglas 生产函数 Y = K^alpha * L^(1-alpha)"""
    return K**alpha * L**(1 - alpha)

def marginal_products(K, L, alpha):
    """计算边际产量"""
    # MPK = α * K^(α-1) * L^(1-α)
    MPK = alpha * (K ** (alpha - 1)) * (L ** (1 - alpha))
    # MPL = (1-α) * K^α * L^(-α)
    MPL = (1 - alpha) * (K ** alpha) * (L ** (-alpha))
    return MPK, MPL

def numeric_evaluate(alpha_val=0.3, K_val=100.0, L_val=50.0):
    """数值计算边际产量"""
    mpk = alpha_val * (K_val ** (alpha_val - 1)) * (L_val ** (1 - alpha_val))
    mpl = (1 - alpha_val) * (K_val ** alpha_val) * (L_val ** (-alpha_val))
    return mpk, mpl

def price_tatonnement(demand_params=(200,1.0), supply_params=(20,0.5), p0=10.0, steps=50, gamma=0.1):
    """简单的瓦尔拉斯拍卖过程：price_{t+1} = p_t + gamma*(D(p_t)-S(p_t))."""
    a, b = demand_params
    c, d = supply_params
    prices = [p0]
    demands = []
    supplies = []
    p = p0
    for t in range(steps):
        D = max(a - b * p, 0.0)
        S = max(c + d * p, 0.0)
        demands.append(D)
        supplies.append(S)
        p = p + gamma * (D - S)
        p = max(p, 0.0)  # 保持价格非负
        prices.append(p)
    return prices, demands, supplies

## 2. 古典经济学假设可视化 (来自 chapter_2_visualizations.py)

In [ ]:
def visualize_production_function():
    """可视化柯布-道格拉斯生产函数"""
    print("生成生产函数图形...")
    
    # 创建资本和劳动的网格
    K = np.linspace(1, 100, 100)
    L = np.linspace(1, 100, 100)
    K_grid, L_grid = np.meshgrid(K, L)
    
    alpha = 0.3
    Y = cobb_douglas(K_grid, L_grid, alpha)
    
    # 3D图
    fig = plt.figure(figsize=(15, 5))
    
    # 3D表面图
    ax1 = fig.add_subplot(131, projection='3d')
    surf = ax1.plot_surface(K_grid, L_grid, Y, cmap='viridis', alpha=0.7)
    ax1.set_xlabel('资本 K')
    ax1.set_ylabel('劳动 L')
    ax1.set_zlabel('产出 Y')
    ax1.set_title('柯布-道格拉斯生产函数\nY = K^α * L^(1-α), α=0.3')
    
    # 等产量线图
    ax2 = fig.add_subplot(132)
    contour = ax2.contour(K, L, Y, levels=15, colors='blue')
    ax2.clabel(contour, inline=True, fontsize=8)
    ax2.set_xlabel('资本 K')
    ax2.set_ylabel('劳动 L')
    ax2.set_title('等产量线图')
    ax2.grid(True)
    
    # 固定劳动，显示产出随资本变化
    ax3 = fig.add_subplot(133)
    L_fixed = 50
    K_range = np.linspace(1, 100, 100)
    Y_vs_K = [cobb_douglas(K_val, L_fixed, alpha) for K_val in K_range]
    ax3.plot(K_range, Y_vs_K, 'b-', linewidth=2, label=f'L={L_fixed}')
    ax3.set_xlabel('资本 K')
    ax3.set_ylabel('产出 Y')
    ax3.set_title(f'固定劳动L={L_fixed}时，Y关于K的变化')
    ax3.grid(True)
    ax3.legend()
    
    plt.tight_layout()
    os.makedirs('results', exist_ok=True)
    plt.savefig('results/production_function.png', dpi=150, bbox_inches='tight')
    plt.close()
    print("生产函数图形已保存至 results/production_function.png")

In [ ]:
def visualize_marginal_products():
    """可视化边际产量递减"""
    print("生成边际产量图形...")
    
    # 参数设置
    alpha = 0.3
    L_val = 50.0
    
    # 资本范围
    K_range = np.linspace(10, 400, 200)
    
    # 计算边际产量
    MPK_vals = []
    MPL_vals = []
    for K in K_range:
        mpk, mpl = numeric_evaluate(alpha_val=alpha, K_val=K, L_val=L_val)
        MPK_vals.append(mpk)
        MPL_vals.append(mpl)
    
    # 绘图
    plt.figure(figsize=(12, 5))
    
    # 子图1：边际产量随资本变化
    plt.subplot(121)
    plt.plot(K_range, MPK_vals, 'r-', linewidth=2, label='MPK (资本边际产量)')
    plt.plot(K_range, MPL_vals, 'b-', linewidth=2, label='MPL (劳动边际产量)')
    plt.xlabel('资本 K')
    plt.ylabel('边际产量')
    plt.title(f'边际产量递减 (α={alpha}, L={L_val})')
    plt.legend()
    plt.grid(True)
    
    # 子图2：资本-劳动比与边际产量关系
    K_L_ratios = [K / L_val for K in K_range]
    plt.subplot(122)
    plt.plot(K_L_ratios, MPK_vals, 'r-', linewidth=2, label='MPK (资本边际产量)')
    plt.plot(K_L_ratios, MPL_vals, 'b-', linewidth=2, label='MPL (劳动边际产量)')
    plt.xlabel('资本-劳动比 K/L')
    plt.ylabel('边际产量')
    plt.title(f'边际产量与资本-劳动比的关系 (α={alpha})')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.savefig('results/marginal_products.png', dpi=150, bbox_inches='tight')
    plt.close()
    print("边际产量图形已保存至 results/marginal_products.png")

In [ ]:
def visualize_supply_demand_equilibrium():
    """可视化供需均衡（价格灵活性）"""
    print("生成供需均衡图形...")
    
    # 需求和供给参数
    demand_params = (200, 1.0)  # D(p) = 200 - 1.0*p
    supply_params = (20, 0.5)   # S(p) = 20 + 0.5*p
    
    # 计算均衡点
    a, b = demand_params  # D(p) = a - b*p
    c, d = supply_params  # S(p) = c + d*p
    # 均衡: a - b*p = c + d*p => p = (a-c)/(b+d)
    p_eq = (a - c) / (b + d)
    q_eq = a - b * p_eq
    
    # 价格范围
    p_range = np.linspace(0, p_eq * 2, 200)
    demand_curve = [max(a - b * p, 0) for p in p_range]
    supply_curve = [max(c + d * p, 0) for p in p_range]
    
    plt.figure(figsize=(12, 5))
    
    # 供需曲线
    plt.subplot(121)
    plt.plot(p_range, demand_curve, 'b-', linewidth=2, label='需求曲线 D(p)')
    plt.plot(p_range, supply_curve, 'r-', linewidth=2, label='供给曲线 S(p)')
    plt.plot([p_eq, p_eq], [0, q_eq], 'k--', alpha=0.5)
    plt.plot([0, p_eq], [q_eq, q_eq], 'k--', alpha=0.5)
    plt.plot(p_eq, q_eq, 'go', markersize=10, label=f'均衡点 ({p_eq:.2f}, {q_eq:.2f})')
    plt.xlabel('价格 p')
    plt.ylabel('数量')
    plt.title('供需均衡与价格灵活性')
    plt.legend()
    plt.grid(True)
    
    # 价格调整过程
    plt.subplot(122)
    prices, demands, supplies = price_tatonnement(demand_params, supply_params, p0=5.0, steps=50, gamma=0.05)
    ts = list(range(len(prices)))
    plt.plot(ts, prices, 'g-', linewidth=2, label='价格 p(t)')
    plt.axhline(y=p_eq, color='k', linestyle='--', alpha=0.7, label=f'均衡价格={p_eq:.2f}')
    plt.xlabel('时间 t')
    plt.ylabel('价格 p')
    plt.title('价格调整至均衡的过程')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.savefig('results/supply_demand_equilibrium.png', dpi=150, bbox_inches='tight')
    plt.close()
    print("供需均衡图形已保存至 results/supply_demand_equilibrium.png")

In [ ]:
def visualize_savings_investment():
    """可视化储蓄-投资均衡"""
    print("生成储蓄-投资均衡图形...")
    
    # 参数：储蓄函数 S(r) = s0 + s1*r，投资函数 I(r) = i0 - i1*r
    s_params = (10, 5)    # S(r) = 10 + 5*r
    i_params = (100, 50)  # I(r) = 100 - 50*r
    
    # 计算均衡点
    s0, s1 = s_params  # S(r) = s0 + s1*r
    i0, i1 = i_params  # I(r) = i0 - i1*r
    # 均衡: s0 + s1*r = i0 - i1*r => r = (i0-s0)/(s1+i1)
    r_eq = (i0 - s0) / (s1 + i1)
    si_eq = s0 + s1 * r_eq
    
    # 利率范围
    r_range = np.linspace(0, min(0.2, r_eq * 2), 200)
    savings_curve = [s0 + s1 * r for r in r_range]
    investment_curve = [max(i0 - i1 * r, 0) for r in r_range]
    
    plt.figure(figsize=(10, 6))
    plt.plot(r_range, savings_curve, 'b-', linewidth=2, label='储蓄 S(r) = s0 + s1*r')
    plt.plot(r_range, investment_curve, 'r-', linewidth=2, label='投资 I(r) = i0 - i1*r')
    plt.plot([r_eq, r_eq], [0, si_eq], 'k--', alpha=0.5)
    plt.plot([0, r_eq], [si_eq, si_eq], 'k--', alpha=0.5)
    plt.plot(r_eq, si_eq, 'go', markersize=10, label=f'均衡点 ({r_eq:.4f}, {si_eq:.2f})')
    plt.xlabel('利率 r')
    plt.ylabel('数量')
    plt.title('储蓄-投资均衡与利率调节机制')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.savefig('results/savings_investment.png', dpi=150, bbox_inches='tight')
    plt.close()
    print("储蓄-投资均衡图形已保存至 results/savings_investment.png")

In [ ]:
def visualize_classical_model_summary():
    """总结古典经济学模型的关键图形"""
    print("生成古典经济学模型总结图形...")
    
    # 创建包含所有关键概念的综合图形
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # 1. 生产函数与边际产量
    alpha = 0.3
    L_val = 50.0
    K_range = np.linspace(10, 200, 100)
    Y_vals = [cobb_douglas(K, L_val, alpha) for K in K_range]
    MPK_vals = []
    for K in K_range:
        mpk, _ = numeric_evaluate(alpha_val=alpha, K_val=K, L_val=L_val)
        MPK_vals.append(mpk)
    
    ax1 = axes[0, 0]
    ax1_twin = ax1.twinx()
    ax1.plot(K_range, Y_vals, 'b-', linewidth=2, label='总产出 Y')
    ax1_twin.plot(K_range, MPK_vals, 'r--', linewidth=2, label='边际产量 MPK')
    ax1.set_xlabel('资本 K')
    ax1.set_ylabel('产出 Y', color='b')
    ax1_twin.set_ylabel('边际产量', color='r')
    ax1.set_title('生产函数与边际产量递减')
    ax1.grid(True)
    
    # 2. 供需均衡
    demand_params = (200, 1.0)
    supply_params = (20, 0.5)
    a, b = demand_params
    c, d = supply_params
    p_eq = (a - c) / (b + d)
    q_eq = a - b * p_eq
    
    p_range = np.linspace(0, p_eq * 2, 200)
    demand_curve = [max(a - b * p, 0) for p in p_range]
    supply_curve = [max(c + d * p, 0) for p in p_range]
    
    ax2 = axes[0, 1]
    ax2.plot(p_range, demand_curve, 'b-', linewidth=2, label='需求 D(p)')
    ax2.plot(p_range, supply_curve, 'r-', linewidth=2, label='供给 S(p)')
    ax2.plot(p_eq, q_eq, 'go', markersize=10, label=f'均衡 ({p_eq:.2f}, {q_eq:.2f})')
    ax2.set_xlabel('价格 p')
    ax2.set_ylabel('数量')
    ax2.set_title('市场出清与价格灵活性')
    ax2.legend()
    ax2.grid(True)
    
    # 3. 储蓄-投资均衡
    s_params = (10, 5)
    i_params = (100, 50)
    s0, s1 = s_params
    i0, i1 = i_params
    r_eq = (i0 - s0) / (s1 + i1)
    si_eq = s0 + s1 * r_eq
    
    r_range = np.linspace(0, min(0.2, r_eq * 2), 200)
    savings_curve = [s0 + s1 * r for r in r_range]
    investment_curve = [max(i0 - i1 * r, 0) for r in r_range]
    
    ax3 = axes[1, 0]
    ax3.plot(r_range, savings_curve, 'b-', linewidth=2, label='储蓄 S(r)')
    ax3.plot(r_range, investment_curve, 'r-', linewidth=2, label='投资 I(r)')
    ax3.plot(r_eq, si_eq, 'go', markersize=10, label=f'均衡 ({r_eq:.4f}, {si_eq:.2f})')
    ax3.set_xlabel('利率 r')
    ax3.set_ylabel('数量')
    ax3.set_title('储蓄-投资与利率调节')
    ax3.legend()
    ax3.grid(True)
    
    # 4. 要素市场均衡（工资由劳动边际产量决定）
    alpha = 0.3
    K_val = 100.0
    L_range = np.linspace(5, 100, 100)
    MPL_vals = []
    wages = []  # 假设工资等于边际产量
    for L in L_range:
        _, mpl = numeric_evaluate(alpha_val=alpha, K_val=K_val, L_val=L)
        MPL_vals.append(mpl)
        wages.append(mpl)  # 在古典模型中，w = MPL
    
    ax4 = axes[1, 1]
    ax4.plot(L_range, MPL_vals, 'r-', linewidth=2, label='劳动边际产量 MPL')
    ax4.plot(L_range, wages, 'g--', linewidth=2, label='工资 w (=MPL)')
    ax4.set_xlabel('劳动 L')
    ax4.set_ylabel('边际产量/工资')
    ax4.set_title('要素定价：工资等于劳动边际产量')
    ax4.legend()
    ax4.grid(True)
    
    plt.tight_layout()
    plt.savefig('results/classical_model_summary.png', dpi=150, bbox_inches='tight')
    plt.close()
    print("古典经济学模型总结图形已保存至 results/classical_model_summary.png")

## 3. 边际产量递减概念可视化 (来自 marginal_diminishing_visualization.py)

In [ ]:
def visualize_diminishing_returns():
    """可视化边际产量递减"""
    print("生成边际产量递减可视化...")
    
    # 参数设置
    alpha = 0.3
    L_val = 50.0  # 固定劳动量
    
    # 资本范围
    K_range = np.linspace(10, 200, 200)
    
    # 计算总产量和边际产量
    Y_vals = [cobb_douglas(K, L_val, alpha) for K in K_range]
    MPK_vals = []
    for K in K_range:
        mpk, _ = marginal_products(K, L_val, alpha)
        MPK_vals.append(mpk)
    
    # 创建图形
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))
    
    # 1. 总产量曲线
    ax1.plot(K_range, Y_vals, 'b-', linewidth=2)
    ax1.set_xlabel('资本 K')
    ax1.set_ylabel('总产出 Y')
    ax1.set_title('总产量曲线（随资本增加）')
    ax1.grid(True)
    
    # 2. 边际产量曲线（递减）
    ax2.plot(K_range, MPK_vals, 'r-', linewidth=2)
    ax2.set_xlabel('资本 K')
    ax2.set_ylabel('资本边际产量 MPK')
    ax2.set_title('边际产量递减（而非负值）')
    ax2.grid(True)
    
    # 3. 平均产量曲线
    APK_vals = [Y/K for Y, K in zip(Y_vals, K_range)]  # 平均资本产量
    ax3.plot(K_range, APK_vals, 'g-', linewidth=2, label='平均产量 APK')
    ax3.plot(K_range, MPK_vals, 'r-', linewidth=2, label='边际产量 MPK')
    ax3.set_xlabel('资本 K')
    ax3.set_ylabel('产量')
    ax3.set_title('平均产量与边际产量比较')
    ax3.legend()
    ax3.grid(True)
    
    # 4. 显示边际产量递减率
    # 计算边际产量的变化率
    delta_MPK = [(MPK_vals[i+1] - MPK_vals[i])/(K_range[i+1] - K_range[i]) for i in range(len(MPK_vals)-1)]
    K_mid = [(K_range[i] + K_range[i+1])/2 for i in range(len(K_range)-1)]
    
    ax4.plot(K_mid, delta_MPK, 'purple', linewidth=2)
    ax4.set_xlabel('资本 K')
    ax4.set_ylabel('边际产量变化率')
    ax4.set_title('边际产量的变化率（始终为负，表示递减）')
    ax4.grid(True)
    
    plt.tight_layout()
    plt.savefig('results/marginal_diminishing_explanation.png', dpi=150, bbox_inches='tight')
    plt.close()
    print("边际产量递减可视化已保存至 results/marginal_diminishing_explanation.png")

In [ ]:
def compare_regions():
    """比较合理与不合理生产区域"""
    print("生成生产区域比较图...")
    
    # 参数设置
    alpha = 0.3
    L_val = 50.0
    K_range = np.linspace(1, 200, 200)
    
    # 计算边际产量
    MPK_vals = []
    for K in K_range:
        mpk, _ = marginal_products(K, L_val, alpha)
        MPK_vals.append(mpk)
    
    plt.figure(figsize=(12, 6))
    
    # 标注不同区域
    plt.plot(K_range, MPK_vals, 'r-', linewidth=2, label='资本边际产量 MPK')
    
    # 合理生产区域（边际产量为正但递减）
    valid_region = [i for i, mpk in enumerate(MPK_vals) if mpk > 0]
    if valid_region:
        start_idx, end_idx = valid_region[0], valid_region[-1]
        plt.axvspan(K_range[start_idx], K_range[end_idx], alpha=0.3, color='green', label='合理生产区域')
    
    plt.xlabel('资本 K')
    plt.ylabel('资本边际产量 MPK')
    plt.title('生产区域划分：合理区域（正边际产量）vs 不合理区域（负边际产量）')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.savefig('results/production_regions.png', dpi=150, bbox_inches='tight')
    plt.close()
    print("生产区域比较图已保存至 results/production_regions.png")

## 4. 运行所有可视化

In [ ]:
# 创建结果目录
os.makedirs('results', exist_ok=True)

# 运行所有可视化函数
visualize_production_function()
visualize_marginal_products()
visualize_supply_demand_equilibrium()
visualize_savings_investment()
visualize_classical_model_summary()
visualize_diminishing_returns()
compare_regions()

## 5. 古典经济学假设总结

In [ ]:
# 显示生成的图像
import matplotlib.image as mpimg

# 创建一个包含所有图像的概览
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# 读取并显示图像
images = [
    'results/production_function.png',
    'results/marginal_products.png',
    'results/supply_demand_equilibrium.png',
    'results/savings_investment.png',
    'results/classical_model_summary.png',
    'results/marginal_diminishing_explanation.png'
]

titles = [
    '生产函数',
    '边际产量递减',
    '供需均衡',
    '储蓄-投资均衡',
    '古典模型综述',
    '边际递减详解'
]

for i, (img_path, title) in enumerate(zip(images[:6], titles)):
    try:
        img = mpimg.imread(img_path)
        row = i // 3
        col = i % 3
        axes[row, col].imshow(img)
        axes[row, col].set_title(title)
        axes[row, col].axis('off')
    except FileNotFoundError:
        axes[row, col].text(0.5, 0.5, f'Image not found:\n{img_path}', 
                           horizontalalignment='center', verticalalignment='center',
                           transform=axes[row, col].transAxes)
        axes[row, col].set_title(title)
        axes[row, col].axis('off')

plt.tight_layout()
plt.show()